In [1]:
from typing import List

def get_chunks(file: str) -> List:
    with open(file, 'r', encoding='UTF-8') as f:
        texts = f.read()
        chunks = texts.split('\n\n')
    return chunks

chunks = get_chunks('doc.md')

for i, chunk in enumerate(chunks):
    print(f'[{i}] {chunk}')

[0] # 福爾摩斯的霧夜之謎
[1] 倫敦的清晨總是籠罩在灰白色的霧氣之中，貝克街 221B 的窗外，馬車的聲音若隱若現。福爾摩斯坐在壁爐旁，手中把玩著他的放大鏡，目光卻緊盯著桌上一封剛送達的信件。
[2] 那封信來自一位名叫艾德蒙·卡特的紡織商人，信中描述了一件離奇的事件：他的工廠在三個夜晚內接連失竊，但倉庫大門與窗戶毫無破壞痕跡，守夜人也聲稱未曾見到任何可疑人物。
[3] 華生醫生推了推眼鏡，讀完信後皺起眉頭，認為這可能只是內部人員所為。然而，福爾摩斯卻注意到信紙邊緣殘留的一抹深藍色染料，與一般書寫用墨水並不相同。
[4] 當天下午，他們來到位於泰晤士河南岸的工廠。福爾摩斯在倉庫地板上蹲下，發現了一串幾乎被清理乾淨的腳印，只在牆角留下微弱的痕跡，顯示出來者行走時刻意踮起腳尖。
[5] 夜幕降臨後，福爾摩斯要求工廠照常運作，並關閉所有對外通道。他自己則躲在二樓辦公室的陰影中，靜靜觀察倉庫的每一個角落。
[6] 接近午夜時分，一陣極輕微的金屬摩擦聲響起。福爾摩斯立刻注意到牆上一塊不起眼的木板被緩緩移開，露出一條狹窄的暗道，正好通往隔壁早已廢棄的染坊。
[7] 嫌犯很快現身——竟是工廠的會計師。他利用職務之便，事先熟悉建築結構，並透過暗道將布料轉移至染坊，再以深藍色染料重新標記，混入合法貨物中出售。
[8] 華生驚訝地問福爾摩斯是如何確定是他。福爾摩斯微微一笑，指出那封信上的染料正是會計師負責管理的特殊貨品，而踮腳行走的腳印，則是為了避免留下熟悉的鞋底花紋。
[9] 案件解決後，艾德蒙·卡特感激地向福爾摩斯致謝，承諾會重新整頓內部管理。工廠的煙囪再次冒起白煙，彷彿一切回到了正軌。
[10] 回到貝克街，福爾摩斯將信件歸檔，淡淡地說道：「在最完美的犯罪中，往往藏著最微小的疏忽。」華生則在筆記本上，默默記下這場霧夜中的推理。


In [2]:
from sentence_transformers import SentenceTransformer


embedding_model = SentenceTransformer('shibing624/text2vec-base-chinese')

def embed_chunk(chunk: str) -> List[float]:
    embedding = embedding_model.encode(chunk, normalize_embeddings=True)
    return embedding.tolist()

embeddings = [embed_chunk(chunk) for chunk in chunks]

print(len(embeddings))
print(len(embeddings[0]))
print(embeddings[0])

f:\AI-Agent\.venv\Lib\site-packages\tqdm\auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


11
768
[-0.008226078934967518, 0.007109357509762049, -0.0006562460330314934, -0.0003389069752302021, 0.006252217572182417, -0.09648339450359344, 0.003804136300459504, -0.018975313752889633, -0.004799388814717531, -0.035134825855493546, -0.02720261551439762, 0.01552243996411562, 0.06891549378633499, -0.011589682660996914, -0.016184428706765175, -0.022077297791838646, 0.003574957139790058, 0.009783562272787094, -0.014636877924203873, -0.056489668786525726, 0.06776077300310135, 0.053585559129714966, -0.05949794873595238, 0.029592270031571388, 0.030398543924093246, -0.020793644711375237, 0.023196181282401085, 0.028200296685099602, 0.01816421002149582, -0.034319669008255005, 0.012072009034454823, -0.006373876240104437, -0.01592906564474106, 0.01752196066081524, -0.02369087189435959, 0.007624173071235418, -0.005673226900398731, -0.043623656034469604, 0.02233724482357502, 0.025636503472924232, 0.014214827679097652, 0.023019753396511078, -0.10165970027446747, -0.018461424857378006, -0.01581074

In [3]:
import chromadb

chromadb_client = chromadb.EphemeralClient()
chromadb_collection = chromadb_client.get_or_create_collection(name="default")

def save_embeddings(chunks: List[str], embeddings: List[List[float]]) -> None:
    for i, (chunk, embedding) in enumerate(zip(chunks, embeddings)):
        chromadb_collection.add(
            documents=[chunk],
            embeddings=[embedding],
            ids=[str(i)]
        )

save_embeddings(chunks, embeddings)

In [4]:
def retrieve(query: str, top_k: int) -> List[str]:
    query_embedding = embed_chunk(query)
    results = chromadb_collection.query(
        query_embeddings=[query_embedding],
        n_results=top_k
    )
    return results['documents'][0]

query = "福爾摩斯是根據哪些線索，判斷竊賊是工廠的會計師，而不是外來的入侵者？"
retrieved_chunks = retrieve(query, 5)

for i, chunk in enumerate(retrieved_chunks):
    print(f"[{i}] {chunk}\n")

[0] 華生驚訝地問福爾摩斯是如何確定是他。福爾摩斯微微一笑，指出那封信上的染料正是會計師負責管理的特殊貨品，而踮腳行走的腳印，則是為了避免留下熟悉的鞋底花紋。

[1] 回到貝克街，福爾摩斯將信件歸檔，淡淡地說道：「在最完美的犯罪中，往往藏著最微小的疏忽。」華生則在筆記本上，默默記下這場霧夜中的推理。

[2] 接近午夜時分，一陣極輕微的金屬摩擦聲響起。福爾摩斯立刻注意到牆上一塊不起眼的木板被緩緩移開，露出一條狹窄的暗道，正好通往隔壁早已廢棄的染坊。

[3] 當天下午，他們來到位於泰晤士河南岸的工廠。福爾摩斯在倉庫地板上蹲下，發現了一串幾乎被清理乾淨的腳印，只在牆角留下微弱的痕跡，顯示出來者行走時刻意踮起腳尖。

[4] 嫌犯很快現身——竟是工廠的會計師。他利用職務之便，事先熟悉建築結構，並透過暗道將布料轉移至染坊，再以深藍色染料重新標記，混入合法貨物中出售。



In [5]:
from sentence_transformers import CrossEncoder

def rerank(query: str, retrieved_chunks: List[str], top_k: int) -> List[str]:
    cross_encoder = CrossEncoder('cross-encoder/mmarco-mMiniLMv2-L12-H384-v1')
    pairs = [(query, chunk) for chunk in retrieved_chunks]
    scores = cross_encoder.predict(pairs)

    scored_chunks = list(zip(retrieved_chunks, scores))
    scored_chunks.sort(key=lambda x: x[1], reverse=True)

    return [chunk for chunk, _ in scored_chunks][:top_k]

reranked_chunks = rerank(query, retrieved_chunks, 3)

for i, chunk in enumerate(reranked_chunks):
    print(f"[{i}] {chunk}\n")

[0] 華生驚訝地問福爾摩斯是如何確定是他。福爾摩斯微微一笑，指出那封信上的染料正是會計師負責管理的特殊貨品，而踮腳行走的腳印，則是為了避免留下熟悉的鞋底花紋。

[1] 嫌犯很快現身——竟是工廠的會計師。他利用職務之便，事先熟悉建築結構，並透過暗道將布料轉移至染坊，再以深藍色染料重新標記，混入合法貨物中出售。

[2] 當天下午，他們來到位於泰晤士河南岸的工廠。福爾摩斯在倉庫地板上蹲下，發現了一串幾乎被清理乾淨的腳印，只在牆角留下微弱的痕跡，顯示出來者行走時刻意踮起腳尖。



In [6]:
import os
from dotenv import load_dotenv
from openai import OpenAI

load_dotenv()
client = OpenAI(
    base_url="https://openrouter.ai/api/v1",
    api_key=os.getenv("API_KEY"),
)


def generate(query: str, chunks: List[str]) -> str:
    prompt = f"""你是一位知識助手，請根據使用者的問題和下列片段產生準確的答案。

使用者問題: {query}

相關片段:
{"\n\n".join(chunks)}

請基於上述內容作答，不要編造資訊。"""

    print(f"{prompt}\n\n---\n")


    completion = client.chat.completions.create(
        model="deepseek/deepseek-r1-0528:free",
        messages=[
            {
                "role": "user",
                "content": prompt
            },
        ],
        temperature=0,
    )
    return completion.choices[0].message.content

answer = generate(query, reranked_chunks)
print(answer)

你是一位知識助手，請根據使用者的問題和下列片段產生準確的答案。

使用者問題: 福爾摩斯是根據哪些線索，判斷竊賊是工廠的會計師，而不是外來的入侵者？

相關片段:
華生驚訝地問福爾摩斯是如何確定是他。福爾摩斯微微一笑，指出那封信上的染料正是會計師負責管理的特殊貨品，而踮腳行走的腳印，則是為了避免留下熟悉的鞋底花紋。

嫌犯很快現身——竟是工廠的會計師。他利用職務之便，事先熟悉建築結構，並透過暗道將布料轉移至染坊，再以深藍色染料重新標記，混入合法貨物中出售。

當天下午，他們來到位於泰晤士河南岸的工廠。福爾摩斯在倉庫地板上蹲下，發現了一串幾乎被清理乾淨的腳印，只在牆角留下微弱的痕跡，顯示出來者行走時刻意踮起腳尖。

請基於上述內容作答，不要編造資訊。

---

根據提供的相關片段，福爾摩斯判斷竊賊是工廠的會計師（而非外來入侵者）的線索主要有兩個，具體如下：

1. **信上的染料線索**：  
   福爾摩斯指出，竊賊留下的信件上使用的染料是工廠內一種「特殊貨品」，而這種染料正是由會計師負責管理的。這意味著只有內部人員（特別是會計師）才能接觸到這種專用染料，外來入侵者不太可能取得或使用它。因此，這條線索直接指向工廠內部人員的涉案。

2. **踮腳行走的腳印線索**：  
   福爾摩斯在倉庫地板上發現了一串幾乎被清理乾淨的腳印，只在牆角留下微弱痕跡。這些腳印顯示竊賊行走時刻意「踮起腳尖」，目的是避免留下完整的鞋底花紋。福爾摩斯推斷，這種行為是為了隱藏竊賊的身份，因為如果留下完整的腳印，其鞋底花紋可能被認出是工廠內部人員（如會計師）的鞋子。外來入侵者通常不需要這樣刻意隱藏鞋底特徵，因為他們並非工廠員工，鞋底花紋不會被熟悉。

這些線索結合起來，表明竊賊是內部人員（會計師），而非外來者。會計師利用職務之便熟悉建築結構和暗道（如片段中所述），但這點是對犯案手法的補充說明，並非福爾摩斯最初判斷的主要線索。以上資訊完全基於提供的片段，未添加任何外部內容。
